In [1]:
import pickle

import numpy as np
from sklearn import model_selection, preprocessing
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'1.13.1'

# Data Preprocessing

In [7]:
!ls -lh ../data/raw/

total 76G
-rw-rw-r--. 1 pughdr g-pughdr 6.8G Feb  3 14:23 all_object_data_in_dictionary_format.pkl
-rw-rw-r--. 1 pughdr g-pughdr 429M Feb 12 12:33 autoscan_features.2.csv
-rw-rw-r--. 1 pughdr g-pughdr  13G Feb  3 14:38 normalized_image_object_data_in_numpy_format.pkl
-rw-rw-r--. 1 pughdr g-pughdr 5.6G Feb  7 15:32 stamps_0.tar
-rw-rw-r--. 1 pughdr g-pughdr 5.6G Feb  7 15:30 stamps_1.tar
-rw-rw-r--. 1 pughdr g-pughdr 5.6G Feb  7 15:31 stamps_2.tar
-rw-rw-r--. 1 pughdr g-pughdr 5.6G Feb  7 15:30 stamps_3.tar
-rw-rw-r--. 1 pughdr g-pughdr 5.6G Feb  7 15:34 stamps_4.tar
-rw-rw-r--. 1 pughdr g-pughdr 5.6G Feb  7 15:31 stamps_5.tar
-rw-rw-r--. 1 pughdr g-pughdr 5.6G Feb  7 15:31 stamps_6.tar
-rw-rw-r--. 1 pughdr g-pughdr 5.6G Feb  7 16:38 stamps_7.tar
-rw-rw-r--. 1 pughdr g-pughdr 5.6G Feb  7 15:30 stamps_8.tar
-rw-rw-r--. 1 pughdr g-pughdr 5.5G Feb  7 16:16 stamps_9.tar


In [3]:
with open("../data/raw/all_object_data_in_dictionary_format.pkl", "rb") as pickled_data:
    all_data = pickle.load(pickled_data)

In [4]:
X, y = all_data["images"], all_data["targets"]

In [5]:
scaler = preprocessing.MinMaxScaler()
Z = scaler.fit_transform(X.reshape(-1, 3 * 51**2))

/home/pughdr/.conda/envs/tensorflow-gpu-tutorials/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
training_features, testing_features, training_target, testing_target = model_selection.train_test_split(Z, y, test_size=0.2)

In [7]:
training_features.shape

(715526, 7803)

In [8]:
testing_features.shape

(178882, 7803)

# Start with a simple DNN

Start with a simple Deep Neural Network (DNN) with a single hidden layer as a benchmark. A simple DNN is able to achieve over 90% accuracy and recall on the test set! Unlike classical ML approaches which require expensive to obtain hand-engineered features, this simple DNN works with the raw image data.

In [20]:
model_fn = keras.models.Sequential([
    keras.layers.Flatten(data_format="channels_first", input_shape=(3, 51, 51)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

_metrics = [
    keras.metrics.BinaryAccuracy(),
    keras.metrics.Recall()
]
model_fn.compile(optimizer="adam", loss="binary_crossentropy", metrics=_metrics)
model_fn.summary()

In [21]:
model_fn.fit(training_features.reshape((-1, 3, 51, 51)), training_target, epochs=2)

Epoch 1/2
715526/715526 [==============================] - 192s 269us/sample - loss: 0.1993 - binary_accuracy: 0.9228 - recall_2: 0.9275
Epoch 2/2
715526/715526 [==============================] - 182s 254us/sample - loss: 0.1669 - binary_accuracy: 0.9358 - recall_2: 0.9410


In [22]:
model_fn.evaluate(testing_features.reshape((-1, 3, 51, 51)), testing_target)

178882/178882 [==============================] - 36s 199us/sample - loss: 0.1834 - binary_accuracy: 0.9294 - recall_2: 0.8952


[0.1834107443779334, 0.9293613, 0.89524364]

# Improve upon DNN by adding convolutions

Show how we can improve performance by adding convolutional layers to our model.

In [64]:
model_fn = keras.models.Sequential([
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), data_format="channels_first", input_shape=(3, 51, 51)),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(pool_size=(2,2), data_format="channels_first"),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), data_format="channels_first"),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(pool_size=(2,2), data_format="channels_first"),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), data_format="channels_first"),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(pool_size=(2,2), data_format="channels_first"),
    keras.layers.Flatten(data_format="channels_first"),
    keras.layers.Dense(128),
    keras.layers.ReLU(),
    keras.layers.Dense(1, activation="sigmoid")
])

_metrics = [
    keras.metrics.BinaryAccuracy(),
    keras.metrics.Recall(),
]
model_fn.compile(optimizer="adam", loss="binary_crossentropy", metrics=_metrics)
model_fn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 16, 49, 49)        448       
_________________________________________________________________
re_lu_17 (ReLU)              (None, 16, 49, 49)        0         
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 16, 24, 24)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 32, 22, 22)        4640      
_________________________________________________________________
re_lu_18 (ReLU)              (None, 32, 22, 22)        0         
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 32, 11, 11)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 64, 9, 9)          18496     
__________

In [65]:
model_fn.fit(training_features.reshape((-1, 3, 51, 51)), training_target, epochs=10)

Epoch 1/10
715526/715526 [==============================] - 268s 375us/sample - loss: 0.1297 - binary_accuracy: 0.9529 - recall_11: 0.9592
Epoch 2/10
715526/715526 [==============================] - 252s 353us/sample - loss: 0.1044 - binary_accuracy: 0.9634 - recall_11: 0.9694
Epoch 3/10
715526/715526 [==============================] - 250s 349us/sample - loss: 0.0986 - binary_accuracy: 0.9654 - recall_11: 0.9714
Epoch 4/10
715526/715526 [==============================] - 249s 348us/sample - loss: 0.0943 - binary_accuracy: 0.9671 - recall_11: 0.9726
Epoch 5/10
715526/715526 [==============================] - 248s 346us/sample - loss: 0.0917 - binary_accuracy: 0.9682 - recall_11: 0.9738
Epoch 6/10
715526/715526 [==============================] - 247s 345us/sample - loss: 0.0899 - binary_accuracy: 0.9689 - recall_11: 0.9744
Epoch 7/10
715526/715526 [==============================] - 247s 345us/sample - loss: 0.0882 - binary_accuracy: 0.9694 - recall_11: 0.9747
Epoch 8/10
715526/715526 [=

In [66]:
model_fn.evaluate(testing_features.reshape((-1, 3, 51, 51)), testing_target)

178882/178882 [==============================] - 27s 153us/sample - loss: 0.0966 - binary_accuracy: 0.9677 - recall_11: 0.9657


[0.09656245221981352, 0.96772176, 0.96567065]

### Improve speed of convergence by adding batch normalization?

In [61]:
model_fn = keras.models.Sequential([
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), data_format="channels_first", input_shape=(3, 51, 51)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(pool_size=(2,2), data_format="channels_first"),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), data_format="channels_first"),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(pool_size=(2,2), data_format="channels_first"),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), data_format="channels_first"),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(pool_size=(2,2), data_format="channels_first"),
    keras.layers.Flatten(data_format="channels_first"),
    keras.layers.Dense(128),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Dense(1, activation="sigmoid")
])

_metrics = [
    keras.metrics.BinaryAccuracy(),
    keras.metrics.Recall(),
]
model_fn.compile(optimizer="adam", loss="binary_crossentropy", metrics=_metrics)
model_fn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 16, 49, 49)        448       
_________________________________________________________________
batch_normalization_v1_16 (B (None, 16, 49, 49)        196       
_________________________________________________________________
re_lu_13 (ReLU)              (None, 16, 49, 49)        0         
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 16, 24, 24)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 32, 22, 22)        4640      
_________________________________________________________________
batch_normalization_v1_17 (B (None, 32, 22, 22)        88        
_________________________________________________________________
re_lu_14 (ReLU)              (None, 32, 22, 22)        0         
__________

In [62]:
model_fn.fit(training_features.reshape((-1, 3, 51, 51)), training_target, epochs=10)

Epoch 1/10
715526/715526 [==============================] - 371s 519us/sample - loss: 0.1370 - binary_accuracy: 0.9500 - recall_10: 0.9561
Epoch 2/10
715526/715526 [==============================] - 350s 490us/sample - loss: 0.1104 - binary_accuracy: 0.9610 - recall_10: 0.9672
Epoch 3/10
715526/715526 [==============================] - 346s 483us/sample - loss: 0.1012 - binary_accuracy: 0.9646 - recall_10: 0.9704
Epoch 4/10
715526/715526 [==============================] - 345s 482us/sample - loss: 0.0955 - binary_accuracy: 0.9666 - recall_10: 0.9721
Epoch 5/10
715526/715526 [==============================] - 349s 487us/sample - loss: 0.0911 - binary_accuracy: 0.9682 - recall_10: 0.9737
Epoch 6/10
715526/715526 [==============================] - 347s 484us/sample - loss: 0.0877 - binary_accuracy: 0.9696 - recall_10: 0.9748
Epoch 7/10
715526/715526 [==============================] - 349s 487us/sample - loss: 0.0849 - binary_accuracy: 0.9705 - recall_10: 0.9757
Epoch 8/10
715526/715526 [=

In [63]:
model_fn.evaluate(testing_features.reshape((-1, 3, 51, 51)), testing_target)

178882/178882 [==============================] - 29s 164us/sample - loss: 0.0890 - binary_accuracy: 0.9697 - recall_10: 0.9764


[0.08902497166420971, 0.96967274, 0.976395]

# Estimator API

Show how we generate an `EstimatorSpec` from our Keras model. Useful starting point for distributed training.

In [36]:
estimator = (keras.estimator
                  .model_to_estimator(model_fn, model_dir="../models/cnn/"))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '../models/cnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f280c1a6780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [42]:
training_input_fn = (tf.estimator
                       .inputs
                       .numpy_input_fn(x=training_features.reshape((-1, 3, 51, 51)),
                                       y=training_target.reshape((-1, 1)),
                                       batch_size=128,
                                       num_epochs=5,
                                       shuffle=True))

In [44]:
estimator.train(input_fn=training_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='../models/cnn/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('../models/cnn/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: conv2d_15/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_15/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_16/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_16/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_17/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_17/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_13/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_13/bias; pr

KeyboardInterrupt: 

In [46]:
evaluation_input_fn = (tf.estimator
                         .inputs.numpy_input_fn(x=testing_features.reshape((-1, 3, 51, 51)),
                                                y=testing_target.reshape((-1, 1)),
                                                num_epochs=1,
                                                shuffle=False))

evaluation_results = estimator.evaluate(input_fn=evaluation_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Starting evaluation at 2019-04-21T13:33:43Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../models/cnn/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-21-13:34:00
INFO:tensorflow:Saving dict for global step 1000: binary_accuracy = 0.96870005, global_step = 1000, loss = 0.0922576, recall_4 = 0.9762923
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: ../models/cnn/model.ckpt-1000


In [47]:
evaluation_results

{'binary_accuracy': 0.96870005,
 'loss': 0.0922576,
 'recall_4': 0.9762923,
 'global_step': 1000}

In [12]:
def model_fn(features: tf.Tensor, labels: tf.Tensor, mode: tf.estimator.ModeKeys) -> tf.estimator.EstimatorSpec:
    """Function builds a DAG and wraps it in an EstimatorSpec"""
    
    # reshape the inputs
    input_layer = tf.reshape(features, [-1, 3, 51, 51])
    
    # convolutional layers
    convolution_layer_1 = tf.layers.conv2d(inputs=input_layer,
                                           filters=32,
                                           kernel_size=(5, 5),
                                           padding="same",
                                           data_format="channels_first",
                                           activation=tf.nn.relu)
    
    pooling_layer_1 = tf.layers.max_pooling2d(inputs=convolution_layer_1,
                                              pool_size=(2, 2),
                                              strides=2,
                                              data_format="channels_first",)
    
    convolution_layer_2 = tf.layers.conv2d(inputs=pooling_layer_1,
                                           filters=64,
                                           kernel_size=(5, 5),
                                           padding="same",
                                           data_format="channels_first",
                                           activation=tf.nn.relu)
    
    pooling_layer_2 = tf.layers.max_pooling2d(inputs=convolution_layer_2,
                                              pool_size=(2, 2),
                                              strides=2,
                                              data_format="channels_first",)
    
    # dense layers
    flatten_layer = tf.layers.flatten(inputs=pooling_layer_2)
    dense_layer = tf.layers.dense(inputs=flatten_layer,
                                  units=1024,
                                  activation=tf.nn.relu)
    dropout_layer = tf.layers.dropout(inputs=dense_layer,
                                      rate=0.45,
                                      training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    # output and loss layers
    logit_layer = tf.layers.dense(inputs=dropout_layer, units=1)
    predicted_labels = tf.argmax(inputs=logit_layer)
    loss = tf.losses.softmax_cross_entropy(labels=labels, logits=logit_layer)
        
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        estimator_spec = tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
    elif mode == tf.estimator.ModeKeys.EVAL:
        accuracy = tf.metrics.accuracy(labels, predicted_labels)
        recall = tf.metrics.recall(labels, predicted_labels)
        f_score = tf.metrics.f_score(labels, predicted_labels)
        auc = tf.metrics.accuracy(labels, predicted_labels)
        eval_metric_ops = {"accuracy": accuracy,
                           "recall": recall,
                           "area_under_curve": auc,
                           "f_score": f_score}
        estimator_spec = tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metric_ops)
    else:
        predictions = {"classes": predicted_labels,
                       "probabilities": tf.nn.sigmoid(logit_layer)}
        estimator_spec = tf.estimatorim.EstimatorSpec(mode, predictions)
    
    return estimator_spec
    
    

# Creating the Estimator

In [10]:
super_nova_estimator = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                              model_dir="../models/super-nova-classifiers/cnn")

# Training the model

In [24]:
tf.estimator.inputs.numpy_input_fn?

In [18]:
training_input_fn = tf.estimator.inputs.numpy_input_fn(x=training_features,
                                                       y=training_labels,
                                                       batch_size=128,
                                                       num_epochs=None,
                                                       shuffle=True)

In [ ]:
super_nova_estimator.train(input_fn=training_input_fn, steps=1000)


# Evaluating the model

In [21]:
evaluation_input_fn = tf.estimator.inputs.numpy_input_fn(x=testing_features,
                                                         y=testing_labels,
                                                         num_epochs=1,
                                                         shuffle=False)

evaluation_results = super_nova_estimator.evaluate(input_fn=evaluation_input_fn)